# 데이터 전처리

In [3]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
df = pd.read_csv('winemag-data-130k-v2.csv', index_col=0)

# 2. 중복 제거 및 국가명 표준화
df = df.drop_duplicates(subset=['description'])
df['country'] = df['country'].str.strip().str.title()

# 3. 대륙(Continent) 매핑: "유럽 와인 말고"와 같은 광범위한 제외 쿼리 대응
country_to_continent = {
    'Italy': 'Europe', 'France': 'Europe', 'Spain': 'Europe', 'Portugal': 'Europe', 'Germany': 'Europe',
    'Austria': 'Europe', 'Greece': 'Europe', 'US': 'North America', 'Canada': 'North America',
    'Argentina': 'South America', 'Chile': 'South America', 'Australia': 'Oceania', 'South Africa': 'Africa'
}
df['continent'] = df['country'].map(country_to_continent).fillna('Others')

# 4. 결측치 처리 (품종별 중앙값)
df['price'] = df['price'].fillna(df.groupby('variety')['price'].transform('median'))
df = df.dropna(subset=['price', 'country', 'variety'])

# 5. [추가] 맛 태그 추출 (Flavor Tags): 텍스트 내 주요 키워드를 메타데이터화
# 에이전트가 "오크 향 나는 거"라고 했을 때 벡터 검색보다 더 정확하게 필터링할 수 있게
flavors = ['oak', 'vanilla', 'berry', 'cherry', 'apple', 'citrus', 'acid', 'spice']
for flavor in flavors:
    df[f'tag_{flavor}'] = df['description'].str.contains(flavor, case=False).astype(int)

# 6. 데이터 필터링
# RAG 성능을 위해 너무 낮은 점수는 제외하되, 충분한 데이터 양을 확보
df_filtered = df[df['points'] >= 88].copy()

# 7. [추가] 임베딩용 통합 텍스트 생성
# 에이전트가 읽을 '컨텐츠'와 검색할 '메타데이터'를 분리하여 저장할 준비
df_filtered['page_content'] = df_filtered.apply(
    lambda row: f"Wine: {row['title']}. Variety: {row['variety']}. Origin: {row['country']}. Description: {row['description']}", 
    axis=1
)

# 8. 최종 데이터 저장
df_filtered.to_csv('wine_data_ready.csv', index=False)
print(f"dataset size: {len(df_filtered)}")

dataset size: 71958


# Pinecone 인덱스 생성
- Index Name: wine-sommelier-agent
- Dimension: 1536
- Metric: cosine
- Cloud/Region: aws / us-east-1

## 메타데이터 스키마

|필드명 (Attribute)|설명 (Description)|타입 (Type)|
|:---------|:--------------:|------:|
|country|와인이 생산된 국가 (예: France, Italy, US)|string|
|continent|와인이 생산된 대륙 (예: Europe, North America)|string|
|points|와인 평점 (80-100 사이의 정수)|integer|
|price|와인 가격 (달러 단위)|float|
|variety|포도 품종 (예: Pinot Noir, Chardonnay)|string|
|tag_oak|오크 향 포함 여부 (1이면 포함, 0이면 미포함)|integer|
|tag_acid|산미 포함 여부 (1이면 포함, 0이면 미포함)|integer|

# Pinecone 데이터 적재

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [2]:
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
import time

# 1. 초기화
pc = Pinecone(api_key=pinecone_api_key)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
index_name = "wine-sommelier-agent"

# 2. 데이터 로드
df = pd.read_csv('wine_data_ready.csv')

c:\githome\새 폴더\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# 3. LangChain Document 객체로 변환
documents = []
for _, row in df.iterrows():
    doc = Document(
        page_content=row['page_content'], # 검색 대상 텍스트
        metadata={
            "title": row['title'],
            "country": row['country'],
            "continent": row['continent'],
            "points": int(row['points']),
            "price": float(row['price']),
            "variety": row['variety'],
            "tag_oak": int(row['tag_oak']),
            "tag_acid": int(row['tag_acid'])
        }
    )
    documents.append(doc)

# 4. Batch Upsert (100개씩 나누어 업로드)
batch_size = 100
for i in range(0, len(documents), batch_size):
    batch = documents[i : i + batch_size]
    PineconeVectorStore.from_documents(
        batch, 
        embeddings, 
        index_name=index_name
    )
    print(f"{i + len(batch)} / {len(documents)} 업로드 완료...")
    time.sleep(1) # API 레이트 리밋 방지

C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100 / 71958 업로드 완료...
200 / 71958 업로드 완료...
300 / 71958 업로드 완료...
400 / 71958 업로드 완료...
500 / 71958 업로드 완료...
600 / 71958 업로드 완료...
700 / 71958 업로드 완료...
800 / 71958 업로드 완료...
900 / 71958 업로드 완료...
1000 / 71958 업로드 완료...
1100 / 71958 업로드 완료...
1200 / 71958 업로드 완료...
1300 / 71958 업로드 완료...
1400 / 71958 업로드 완료...
1500 / 71958 업로드 완료...
1600 / 71958 업로드 완료...
1700 / 71958 업로드 완료...
1800 / 71958 업로드 완료...
1900 / 71958 업로드 완료...
2000 / 71958 업로드 완료...
2100 / 71958 업로드 완료...
2200 / 71958 업로드 완료...
2300 / 71958 업로드 완료...
2400 / 71958 업로드 완료...
2500 / 71958 업로드 완료...
2600 / 71958 업로드 완료...
2700 / 71958 업로드 완료...
2800 / 71958 업로드 완료...
2900 / 71958 업로드 완료...
3000 / 71958 업로드 완료...
3100 / 71958 업로드 완료...
3200 / 71958 업로드 완료...
3300 / 71958 업로드 완료...
3400 / 71958 업로드 완료...
3500 / 71958 업로드 완료...
3600 / 71958 업로드 완료...
3700 / 71958 업로드 완료...
3800 / 71958 업로드 완료...
3900 / 71958 업로드 완료...
4000 / 71958 업로드 완료...
4100 / 71958 업로드 완료...
4200 / 71958 업로드 완료...
4300 / 71958 업로드 완료...
4400 / 71958 업로드 완료.

# Metadata Field Info 정의

In [3]:
from langchain.chains.query_constructor.base import AttributeInfo

# 에이전트에게 가르칠 메타데이터 스펙
metadata_field_info = [
    AttributeInfo(
        name="country",
        description="The country where the wine was produced (e.g., Italy, France, US)",
        type="string",
    ),
    AttributeInfo(
        name="continent",
        description="The continent where the wine was produced (e.g., Europe, North America)",
        type="string",
    ),
    AttributeInfo(
        name="points",
        description="The rating of the wine on a scale of 80-100",
        type="integer",
    ),
    AttributeInfo(
        name="price",
        description="The price of the wine in US dollars",
        type="float",
    ),
    AttributeInfo(
        name="variety",
        description="The type of grapes used for the wine (e.g., Pinot Noir, Chardonnay)",
        type="string",
    ),
    AttributeInfo(
        name="tag_oak",
        description="Whether the wine has oaky flavors. 1 if true, 0 if false",
        type="integer",
    ),
]

document_content_description = "Brief description of the wine's taste, aroma, and characteristics"

## Retriever 생성

In [4]:
from langchain_openai import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_pinecone import PineconeVectorStore

# 1. LLM 설정 (쿼리 생성용)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. 벡터 스토어 연결 (이미 업로드된 인덱스 활용)
vectorstore = PineconeVectorStore(
    index_name="wine-sommelier-agent", 
    embedding=embeddings # 앞서 정의한 OpenAIEmbeddings
)

# 3. Self-Query Retriever 초기화
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True # 에이전트가 쿼리를 어떻게 변환하는지 로그를 보기 위해 True
)

In [5]:
# 1. 리트리버 가동
# (위에서 작성한 metadata_field_info와 document_content_description이 정의되어 있어야 합니다.)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True 
)

# 2. 첫 번째 질문 던지기
query = "프랑스 와인 말고 30달러 이하인 가성비 좋은 레드 와인 추천해줘"
response = retriever.invoke(query)

# 3. 결과 확인
for doc in response:
    print(f"와인: {doc.metadata['title']} | 국가: {doc.metadata['country']} | 가격: ${doc.metadata['price']}")

와인: Firesteed 2013 Pinot Noir (Oregon) | 국가: Us | 가격: $17.0
와인: Northstar 2006 Stella Maris Red Red (Columbia Valley (WA)) | 국가: Us | 가격: $29.0
와인: Roederer Estate NV Brut Rosé Sparkling (Anderson Valley) | 국가: Us | 가격: $27.0
와인: Garnet 2012 Rodgers Creek Vineyard Pinot Noir (Sonoma Coast) | 국가: Us | 가격: $30.0


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 템플릿 설계 (소믈리에 페르소나)
template = """당신은 세계 최고의 전문 와인 소믈리에입니다. 
제공된 와인 정보를 바탕으로 사용자의 요청에 친절하고 전문적으로 답해주세요.
추천할 때는 와인의 이름, 지역, 특징, 그리고 왜 이 와인을 선택했는지 이유를 포함해주세요.

검색된 와인 정보:
{context}

사용자 질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 2. RAG 체인 구축
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 3. 최종 테스트
final_response = rag_chain.invoke("프랑스 와인 말고 30달러 이하인 가성비 좋은 레드 와인 추천해줘")
print(final_response)

프랑스 와인을 제외하고 30달러 이하의 가성비 좋은 레드 와인을 추천해드리겠습니다. 

**추천 와인: Northstar 2006 Stella Maris Red Red (Columbia Valley, WA)**  
- **가격:** 29.00달러  
- **포인트:** 91점  
- **특징:** 이 레드 블렌드는 생동감 넘치는 베리 향과 상큼한 산미가 특징입니다. 부드럽고 깊이 있는 맛이 느껴지며, 모카와 감초의 힌트가 마무리에서 더해져 풍부한 맛을 제공합니다. 가격 대비 뛰어난 품질로, 다른 Northstar 와인들과 비교해도 손색이 없습니다.

**추천 이유:** Northstar의 Stella Maris는 가격이 저렴하면서도 뛰어난 품질을 자랑합니다. 다양한 음식과 잘 어울리며, 특히 바비큐나 파스타와 함께 즐기기 좋습니다. 이 와인은 가성비가 뛰어나면서도 고급스러운 맛을 경험할 수 있어, 레드 와인을 좋아하시는 분들께 강력히 추천드립니다.


In [7]:
# 수정된 프롬프트 템플릿
template = """당신은 데이터 분석 역량을 갖춘 '인사이트 소믈리에'입니다. 
제공된 와인 데이터를 체계적으로 분석하여 사용자의 요청에 최적화된 제안을 합니다.

### [분석 지침]
1. **데이터 접지(Grounding):** 반드시 제공된 '검색된 와인 정보' 내의 데이터만 사용하세요. 없는 정보를 지어내지 마세요.
2. **가성비 분석:** 와인의 '평점(Points)'을 '가격(Price)'으로 나눈 가치를 고려하여, 왜 이 와인이 경제적으로도 우수한 선택인지 설명하세요.
3. **배제 로직 확인:** 사용자가 제외를 요청한 조건(예: 특정 국가)이 완벽히 지켜졌는지 최종 검토하세요.

### [출력 구조]
- **요약:** 사용자의 요청 사항을 데이터 관점에서 재정의 (예: "프랑스 제외, $30 이하 레드 와인 분석 결과")
- **추천 리스트:** - 와인명 (국가/지역) | 평점: XX점 | 가격: $XX
  - 💡 **인사이트:** 맛의 특징과 이 와인이 선정된 논리적 이유
- **소믈리에의 제언:** 추천된 와인들을 즐기는 순서나 페어링 팁

검색된 와인 정보:
{context}

사용자 질문: {question}

답변:"""

# 나머지 체인 로직은 동일하게 유지
prompt = ChatPromptTemplate.from_template(template)

# 2. RAG 체인 구축
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 3. 최종 테스트
final_response = rag_chain.invoke("프랑스 와인 말고 30달러 이하인 가성비 좋은 레드 와인 추천해줘")
print(final_response)

### 요약
"프랑스 와인 제외, $30 이하 가성비 좋은 레드 와인 분석 결과"

### 추천 리스트
- **Firesteed 2013 Pinot Noir (Oregon)** | 평점: 88점 | 가격: $17
  - 💡 **인사이트:** 이 Pinot Noir는 오리건의 남부 지역에서 생산되어, 신선한 딸기와 타르트 라즈베리의 맛이 잘 드러납니다. 가격 대비 뛰어난 품질을 자랑하며, 풀바디의 과일 풍미가 매력적입니다. 가성비가 뛰어난 선택입니다.

- **Northstar 2006 Stella Maris Red Red (Columbia Valley (WA))** | 평점: 91점 | 가격: $29
  - 💡 **인사이트:** 이 레드 블렌드는 생동감 있는 산미와 함께 풍부한 베리 맛이 특징입니다. 가격이 다른 Northstar 와인들보다 저렴하지만, 그 품질은 결코 뒤지지 않습니다. 가성비가 매우 우수한 선택입니다.

- **Garnet 2012 Rodgers Creek Vineyard Pinot Noir (Sonoma Coast)** | 평점: 90점 | 가격: $30
  - 💡 **인사이트:** 이 Pinot Noir는 다크 베리와 체리의 농축된 맛이 특징이며, 스모키한 풍미와 함께 흥미로운 스피어민트의 뒷맛이 매력적입니다. 2012년의 뛰어난 수확 덕분에 가격 대비 훌륭한 선택입니다.

### 소믈리에의 제언
추천된 와인들은 각각의 독특한 맛과 향을 가지고 있으므로, 다양한 음식과 페어링하여 즐기기를 권장합니다. 

- **Firesteed Pinot Noir**는 그릴에 구운 닭고기나 연어와 잘 어울립니다.
- **Northstar Stella Maris**는 스테이크나 양고기와 함께하면 그 풍미가 더욱 강조됩니다.
- **Garnet Pinot Noir**는 파스타 요리나 피자와 함께 즐기면 좋습니다.

이 와인들을 순서대로 시음하며 각 와인의 특징을 비교해보는 것도 좋은 경험이 될 것입니다.
